In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sys

In [17]:
class DCGAN():
    def __init__(self, rows, cols, channels, z = 10):
        # Input shape
        self.img_rows = rows
        self.img_cols = cols
        self.channels = channels
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = z
        
        optimizer = tf.keras.optimizers.Adam(lr = 0.0000001, beta_1 = 0.5)
        
        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss = 'binary_crossentropy',
                                   optimizer = optimizer,
                                   metrics = ['accuracy'])
        
        # Build the generator
        self.generator = self.build_generator()
        
        # The generator takes noise as input and generates imgs
        z = tf.keras.layers.Input(shape = (self.latent_dim,))
        img = self.generator(z)
        
        # For the combined model we will only train the generator
        self.discriminator.trainable = False
        
        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)
        
        # The combined model (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = tf.keras.models.Model(z, valid)
        self.combined.compile(loss = 'binary_crossentropy',
                              optimizer = optimizer)
        
    def build_generator(self):
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(128 * 7 * 7, activation = 'relu', input_dim = self.latent_dim),
            tf.keras.layers.Reshape((7, 7, 128)),
            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same'),
            tf.keras.layers.BatchNormalization(momentum = 0.8),
            tf.keras.layers.Activation(activation='relu'),
            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same'),
            tf.keras.layers.BatchNormalization(momentum = 0.8),
            tf.keras.layers.Activation(activation = 'relu'),
            tf.keras.layers.Conv2D(self.channels, kernel_size = 3, padding = 'same'),
            tf.keras.layers.Activation(activation = 'tanh')
        ])
        model.summary()
        noise = tf.keras.layers.Input(shape = (self.latent_dim,))
        img = model(noise)
        return tf.keras.models.Model(noise, img)
    
    def build_discriminator(self):
        model = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, strides = 2,
                                   input_shape = self.img_shape, padding = 'same'),
            tf.keras.layers.LeakyReLU(alpha = 0.2),
            tf.keras.layers.Dropout(rate = 0.25),
            tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, strides = 2, padding = 'same'),
            tf.keras.layers.ZeroPadding2D(padding = ((0,1), (0,1))),
            tf.keras.layers.BatchNormalization(momentum = 0.8),
            tf.keras.layers.LeakyReLU(alpha = 0.2),
            tf.keras.layers.Dropout(rate = 0.25),
            tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, strides = 2, padding = 'same'),
            tf.keras.layers.BatchNormalization(momentum = 0.8),
            tf.keras.layers.LeakyReLU(alpha = 0.2),
            tf.keras.layers.Dropout(rate = 0.25),
            tf.keras.layers.Conv2D(filters = 256, kernel_size = 3, strides = 2, padding = 'same'),
            tf.keras.layers.BatchNormalization(momentum = 0.8),
            tf.keras.layers.LeakyReLU(alpha = 0.2),
            tf.keras.layers.Dropout(rate = 0.25),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units = 1, activation = 'sigmoid')
        ])
        model.summary()
        img = tf.keras.layers.Input(shape = (self.img_shape))
        validity = model(img)
        return tf.keras.models.Model(img, validity)
    
    def train(self, epochs, batch_size = 256, save_interval = 50):
        # Load the dataset
        (x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
        
        # Rescale -1 to 1
        x_train = x_train / 127.5 - 1.
        x_train = np.expand_dims(x_train, axis = 3)
        
        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        for epoch in range(epochs):
            # -------------------
            # Train Discriminator
            # -------------------
            
            # Select a random half of images
            idx = np.random.randint(0, x_train.shape[0], batch_size)
            imgs = x_train[idx]
            
            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)
            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            # -------------------
            # Train Generator
            # -------------------
            
            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)
            
            # Plot the progress
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            
            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)
    
    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :, :, 0], cmap = 'gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig('dcgan_mnist_%d.png' % epoch)
        plt.close()

In [18]:
dcgan = DCGAN(28, 28, 1)
dcgan.train(epochs = 5000, batch_size = 256, save_interval = 50)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_6 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_28 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 8, 8, 64)        

65 [D loss: 0.957799, acc.: 43.75%] [G loss: 0.533966]
66 [D loss: 0.930510, acc.: 43.95%] [G loss: 0.600925]
67 [D loss: 0.940466, acc.: 46.48%] [G loss: 0.589962]
68 [D loss: 0.946548, acc.: 46.48%] [G loss: 0.601070]
69 [D loss: 1.011139, acc.: 43.75%] [G loss: 0.571931]
70 [D loss: 0.983560, acc.: 42.38%] [G loss: 0.556610]
71 [D loss: 0.994947, acc.: 46.88%] [G loss: 0.626135]
72 [D loss: 0.978224, acc.: 44.73%] [G loss: 0.551122]
73 [D loss: 0.912691, acc.: 48.24%] [G loss: 0.566250]
74 [D loss: 1.014188, acc.: 42.38%] [G loss: 0.614631]
75 [D loss: 0.969553, acc.: 43.16%] [G loss: 0.567475]
76 [D loss: 0.968632, acc.: 44.73%] [G loss: 0.562077]
77 [D loss: 1.003165, acc.: 38.48%] [G loss: 0.547013]
78 [D loss: 0.997342, acc.: 45.70%] [G loss: 0.534306]
79 [D loss: 0.960903, acc.: 43.55%] [G loss: 0.522523]
80 [D loss: 0.953656, acc.: 46.09%] [G loss: 0.566457]
81 [D loss: 0.980233, acc.: 43.16%] [G loss: 0.531565]
82 [D loss: 0.986342, acc.: 44.53%] [G loss: 0.557231]
83 [D loss

212 [D loss: 0.941463, acc.: 44.92%] [G loss: 0.551365]
213 [D loss: 0.938018, acc.: 45.31%] [G loss: 0.559748]
214 [D loss: 0.953788, acc.: 45.90%] [G loss: 0.534141]
215 [D loss: 0.923413, acc.: 45.90%] [G loss: 0.564662]
216 [D loss: 0.932127, acc.: 48.24%] [G loss: 0.581519]
217 [D loss: 0.954647, acc.: 43.95%] [G loss: 0.620285]
218 [D loss: 0.945945, acc.: 47.07%] [G loss: 0.579787]
219 [D loss: 0.983674, acc.: 44.53%] [G loss: 0.583220]
220 [D loss: 0.964170, acc.: 44.73%] [G loss: 0.555290]
221 [D loss: 0.929023, acc.: 46.29%] [G loss: 0.596104]
222 [D loss: 0.945323, acc.: 43.55%] [G loss: 0.619202]
223 [D loss: 0.955481, acc.: 42.38%] [G loss: 0.558287]
224 [D loss: 0.933408, acc.: 47.07%] [G loss: 0.591551]
225 [D loss: 0.895908, acc.: 47.66%] [G loss: 0.571649]
226 [D loss: 0.919571, acc.: 45.70%] [G loss: 0.586971]
227 [D loss: 0.938171, acc.: 45.12%] [G loss: 0.578677]
228 [D loss: 0.967460, acc.: 42.97%] [G loss: 0.643995]
229 [D loss: 0.961852, acc.: 43.95%] [G loss: 0.

359 [D loss: 0.925010, acc.: 44.92%] [G loss: 0.625272]
360 [D loss: 0.969437, acc.: 46.09%] [G loss: 0.554375]
361 [D loss: 0.929745, acc.: 46.88%] [G loss: 0.539719]
362 [D loss: 0.882908, acc.: 49.61%] [G loss: 0.561882]
363 [D loss: 0.934816, acc.: 45.70%] [G loss: 0.574881]
364 [D loss: 0.939473, acc.: 44.34%] [G loss: 0.584564]
365 [D loss: 0.936969, acc.: 45.70%] [G loss: 0.553081]
366 [D loss: 0.913433, acc.: 47.07%] [G loss: 0.599219]
367 [D loss: 0.951410, acc.: 47.46%] [G loss: 0.599974]
368 [D loss: 0.910717, acc.: 47.66%] [G loss: 0.561225]
369 [D loss: 0.951061, acc.: 43.36%] [G loss: 0.558215]
370 [D loss: 0.922880, acc.: 47.46%] [G loss: 0.594296]
371 [D loss: 0.913905, acc.: 46.88%] [G loss: 0.573131]
372 [D loss: 0.927939, acc.: 47.46%] [G loss: 0.600693]
373 [D loss: 0.927299, acc.: 45.90%] [G loss: 0.594865]
374 [D loss: 0.900532, acc.: 49.61%] [G loss: 0.609672]
375 [D loss: 0.950653, acc.: 42.77%] [G loss: 0.619220]
376 [D loss: 0.933693, acc.: 47.85%] [G loss: 0.

506 [D loss: 0.883342, acc.: 47.27%] [G loss: 0.587749]
507 [D loss: 0.904057, acc.: 47.46%] [G loss: 0.642088]
508 [D loss: 0.902902, acc.: 50.98%] [G loss: 0.603057]
509 [D loss: 0.940302, acc.: 47.66%] [G loss: 0.599441]
510 [D loss: 0.946932, acc.: 48.05%] [G loss: 0.626161]
511 [D loss: 0.907786, acc.: 49.61%] [G loss: 0.626266]
512 [D loss: 0.914329, acc.: 47.27%] [G loss: 0.600639]
513 [D loss: 0.911756, acc.: 47.85%] [G loss: 0.623955]
514 [D loss: 0.924516, acc.: 46.09%] [G loss: 0.579384]
515 [D loss: 0.902941, acc.: 50.78%] [G loss: 0.608455]
516 [D loss: 0.898857, acc.: 48.44%] [G loss: 0.631417]
517 [D loss: 0.886523, acc.: 48.44%] [G loss: 0.616022]
518 [D loss: 0.925843, acc.: 46.88%] [G loss: 0.608889]
519 [D loss: 0.922177, acc.: 48.05%] [G loss: 0.631358]
520 [D loss: 0.837002, acc.: 50.39%] [G loss: 0.643034]
521 [D loss: 0.890456, acc.: 48.05%] [G loss: 0.602496]
522 [D loss: 0.914033, acc.: 50.98%] [G loss: 0.570122]
523 [D loss: 0.944793, acc.: 45.31%] [G loss: 0.

...

4404 [D loss: 0.620025, acc.: 64.06%] [G loss: 0.797189]
4405 [D loss: 0.614877, acc.: 69.34%] [G loss: 0.764220]
4406 [D loss: 0.614860, acc.: 67.19%] [G loss: 0.793760]
4407 [D loss: 0.626700, acc.: 66.02%] [G loss: 0.733056]
4408 [D loss: 0.581980, acc.: 67.38%] [G loss: 0.761677]
4409 [D loss: 0.600716, acc.: 68.16%] [G loss: 0.796853]
4410 [D loss: 0.625975, acc.: 66.80%] [G loss: 0.782250]
4411 [D loss: 0.600197, acc.: 68.75%] [G loss: 0.759205]
4412 [D loss: 0.636767, acc.: 65.04%] [G loss: 0.790597]
4413 [D loss: 0.620097, acc.: 65.43%] [G loss: 0.812971]
4414 [D loss: 0.633141, acc.: 63.48%] [G loss: 0.770946]
4415 [D loss: 0.631706, acc.: 66.60%] [G loss: 0.762155]
4416 [D loss: 0.612400, acc.: 65.43%] [G loss: 0.811974]
4417 [D loss: 0.660175, acc.: 62.30%] [G loss: 0.729149]
4418 [D loss: 0.660831, acc.: 66.02%] [G loss: 0.782629]
4419 [D loss: 0.615683, acc.: 69.14%] [G loss: 0.830338]
4420 [D loss: 0.612113, acc.: 66.60%] [G loss: 0.805624]
4421 [D loss: 0.650541, acc.: 6

4548 [D loss: 0.595534, acc.: 68.16%] [G loss: 0.813483]
4549 [D loss: 0.628570, acc.: 64.84%] [G loss: 0.813737]
4550 [D loss: 0.602050, acc.: 67.77%] [G loss: 0.802447]
4551 [D loss: 0.615166, acc.: 66.80%] [G loss: 0.773005]
4552 [D loss: 0.603697, acc.: 66.02%] [G loss: 0.798642]
4553 [D loss: 0.577500, acc.: 70.90%] [G loss: 0.840230]
4554 [D loss: 0.597525, acc.: 67.38%] [G loss: 0.773713]
4555 [D loss: 0.609414, acc.: 66.99%] [G loss: 0.757542]
4556 [D loss: 0.628988, acc.: 65.82%] [G loss: 0.722140]
4557 [D loss: 0.635065, acc.: 65.62%] [G loss: 0.735191]
4558 [D loss: 0.599054, acc.: 68.55%] [G loss: 0.820261]
4559 [D loss: 0.625515, acc.: 66.02%] [G loss: 0.793099]
4560 [D loss: 0.651917, acc.: 64.45%] [G loss: 0.825720]
4561 [D loss: 0.622150, acc.: 68.16%] [G loss: 0.790354]
4562 [D loss: 0.639973, acc.: 66.99%] [G loss: 0.829094]
4563 [D loss: 0.614782, acc.: 65.82%] [G loss: 0.783664]
4564 [D loss: 0.665848, acc.: 64.84%] [G loss: 0.832895]
4565 [D loss: 0.620675, acc.: 6

4692 [D loss: 0.631321, acc.: 64.84%] [G loss: 0.792131]
4693 [D loss: 0.615345, acc.: 67.97%] [G loss: 0.778081]
4694 [D loss: 0.604287, acc.: 66.60%] [G loss: 0.858414]
4695 [D loss: 0.597244, acc.: 67.38%] [G loss: 0.759204]
4696 [D loss: 0.573109, acc.: 69.53%] [G loss: 0.772805]
4697 [D loss: 0.611912, acc.: 66.21%] [G loss: 0.807027]
4698 [D loss: 0.609740, acc.: 68.55%] [G loss: 0.762246]
4699 [D loss: 0.600957, acc.: 69.14%] [G loss: 0.809032]
4700 [D loss: 0.608806, acc.: 67.58%] [G loss: 0.817240]
4701 [D loss: 0.634837, acc.: 65.23%] [G loss: 0.775572]
4702 [D loss: 0.628406, acc.: 66.99%] [G loss: 0.824206]
4703 [D loss: 0.613602, acc.: 65.62%] [G loss: 0.845375]
4704 [D loss: 0.630621, acc.: 63.87%] [G loss: 0.802110]
4705 [D loss: 0.615443, acc.: 66.21%] [G loss: 0.789434]
4706 [D loss: 0.593627, acc.: 68.36%] [G loss: 0.793239]
4707 [D loss: 0.577364, acc.: 68.36%] [G loss: 0.833209]
4708 [D loss: 0.591470, acc.: 67.38%] [G loss: 0.801554]
4709 [D loss: 0.587625, acc.: 6

4836 [D loss: 0.595942, acc.: 67.58%] [G loss: 0.829258]
4837 [D loss: 0.587765, acc.: 67.97%] [G loss: 0.786963]
4838 [D loss: 0.632701, acc.: 63.87%] [G loss: 0.796696]
4839 [D loss: 0.604261, acc.: 67.19%] [G loss: 0.803143]
4840 [D loss: 0.619612, acc.: 63.67%] [G loss: 0.732346]
4841 [D loss: 0.578720, acc.: 69.34%] [G loss: 0.832176]
4842 [D loss: 0.622136, acc.: 65.23%] [G loss: 0.782081]
4843 [D loss: 0.669061, acc.: 62.30%] [G loss: 0.811047]
4844 [D loss: 0.607300, acc.: 67.19%] [G loss: 0.851619]
4845 [D loss: 0.630375, acc.: 66.60%] [G loss: 0.785554]
4846 [D loss: 0.635420, acc.: 64.84%] [G loss: 0.792919]
4847 [D loss: 0.657361, acc.: 66.02%] [G loss: 0.757547]
4848 [D loss: 0.590940, acc.: 68.16%] [G loss: 0.780370]
4849 [D loss: 0.581083, acc.: 69.92%] [G loss: 0.796022]
4850 [D loss: 0.604559, acc.: 66.21%] [G loss: 0.734505]
4851 [D loss: 0.605267, acc.: 67.58%] [G loss: 0.770377]
4852 [D loss: 0.600827, acc.: 66.80%] [G loss: 0.749965]
4853 [D loss: 0.639281, acc.: 6

4980 [D loss: 0.606351, acc.: 66.80%] [G loss: 0.788582]
4981 [D loss: 0.596210, acc.: 66.21%] [G loss: 0.836199]
4982 [D loss: 0.605450, acc.: 67.19%] [G loss: 0.821231]
4983 [D loss: 0.627312, acc.: 64.45%] [G loss: 0.756495]
4984 [D loss: 0.612523, acc.: 67.19%] [G loss: 0.823856]
4985 [D loss: 0.626696, acc.: 64.45%] [G loss: 0.774720]
4986 [D loss: 0.610676, acc.: 67.19%] [G loss: 0.824805]
4987 [D loss: 0.604757, acc.: 68.95%] [G loss: 0.769527]
4988 [D loss: 0.635138, acc.: 65.82%] [G loss: 0.834344]
4989 [D loss: 0.651963, acc.: 64.65%] [G loss: 0.789188]
4990 [D loss: 0.584718, acc.: 66.80%] [G loss: 0.756044]
4991 [D loss: 0.572639, acc.: 67.77%] [G loss: 0.864956]
4992 [D loss: 0.585427, acc.: 68.95%] [G loss: 0.781528]
4993 [D loss: 0.599206, acc.: 67.77%] [G loss: 0.821484]
4994 [D loss: 0.626082, acc.: 64.84%] [G loss: 0.807102]
4995 [D loss: 0.615374, acc.: 66.41%] [G loss: 0.818584]
4996 [D loss: 0.632018, acc.: 65.62%] [G loss: 0.810084]
4997 [D loss: 0.569194, acc.: 6